https://github.com/ugent-korea/pytorch-unet-segmentation#postprocessing

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import torch
import numpy as np
import sys
from PIL import Image
from torchvision import transforms
sys.path.append("../pytorch-unet-segmentation-master/src/")

In [ ]:
# Modelo y definiciones

model = torch.load('../pytorch-unet-segmentation-master/model_epoch_440.pwf', 
                   map_location=torch.device('cpu')).module.cpu()
model.eval();

r = c = 190
w = h = 388

my_transforms = transforms.Compose([transforms.Pad(padding=(572-388)//2, padding_mode='reflect'),
                                    transforms.ToTensor()])

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Importar imágenes
img = cv2.imread('0.png', cv2.IMREAD_GRAYSCALE)[r:r+h, c:c+w]
label = cv2.imread('0_mask.png', cv2.IMREAD_UNCHANGED)[r:r+h, c:c+w, 3] == 0
fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)
ax[0, 0].imshow(img, cmap=plt.cm.Greys_r)
ax[0, 0].set_title('Original')
ax[1, 0].imshow(label, cmap=plt.cm.Greys_r)
ax[1, 0].set_title('Máscara')

# Preprocesamiento con opencv
img = cv2.equalizeHist(img)
#img = denoise_image(img, 30)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel=np.ones((3,3),np.uint8), iterations = 4)
ax[0, 1].imshow(img, cmap=plt.cm.Greys_r)
ax[0, 1].set_title('Preprocesada')

# Transformar con torch
im_pil = Image.fromarray(img)
img_tensor = my_transforms(im_pil)
label_tensor = torch.from_numpy(label.astype('int'))

# Evaluar con U-net
with torch.no_grad():
    output = model.forward(img_tensor.unsqueeze(0))
    loss = criterion(output, label_tensor.unsqueeze(0)).item()
    preds = torch.argmax(output, dim=1).float().squeeze(0).numpy()
    
ax[1, 1].imshow(preds , cmap=plt.cm.Greys_r)
ax[1, 1].set_title(str(loss));